<a href="https://colab.research.google.com/github/joanneim/code_samples/blob/main/NLP_T5Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine-tuning T5 from the Huggingface Library Simple Transformers

In [ ]:
#!pip install simpletransformers
import pandas as pd
from simpletransformers.t5 import T5Model
from pprint import pprint
import logging

In [ ]:
# Making sure the environment is set up correctly for anyone running this notebook
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import datetime as datetime

**Getting the Data**

In [ ]:
with open("../data/raw/yelp_processedUTF8.csv") as f:
    print(f)

<_io.TextIOWrapper name='../data/raw/yelp_processedUTF8.csv' mode='r' encoding='UTF-8'>


In [ ]:
from tensorflow import keras
from bert import modeling
import os
import re
import pandas as pd
print(os.getcwd())
def download_and_load_dataset(force_download=True):
  return pd.read_csv("../data/raw/yelp_processedUTF8.csv", encoding = 'UTF-8')


/Users/Joanneim/Dropbox (MIT)/Work/github/6862_Fake_Review_Detection/code


In [ ]:
reviews = download_and_load_dataset()

**Processing the data**

In [ ]:
reviews = reviews[['reviewText', 'fakeLabel']]
def refinereviewText(row):
    return row['reviewText'].lower()

def refinefakeLabel(row):
    if row['fakeLabel'] == -1:
        return 0
    else:
        return 1

reviews = reviews.dropna()
reviews['reviewText'] = reviews.apply(refinereviewText, axis=1)
reviews['fakeLabel'] = reviews.apply(refinefakeLabel, axis=1)

df_zeros = reviews[reviews['fakeLabel'] == 0]
df_ones = reviews[reviews['fakeLabel'] == 1].sample(80466)
df_combined = df_zeros.append(df_ones)
df_combined = df_combined.sample(frac=1).reset_index(drop=True)
df_smaller = df_combined.sample(frac=0.1).reset_index(drop=True)
df_smaller

reviews = df_smaller.copy()
reviews = reviews.rename(columns={"reviewText": "review", "fakeLabel": "deceptive"})
reviews.deceptive = reviews.deceptive.astype(str)

In [ ]:
# Check that label is a string
print(reviews.deceptive[0])
type(reviews.deceptive[0])

1


str

In [ ]:
reviews.head(2)

,review,deceptive
0,"the first time i went to geronimo, the service...",1
1,"i miss my msg! ok, make that salt. the touchsc...",0


Everything until this cell has been run of the mill data processing and setup. Now, we split OpSpam into 80/20 train/test, and define the labels we wish to classify.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

cross_num = 4
splitter = StratifiedShuffleSplit(n_splits=5, random_state=910, test_size=0.2)
labels = [str(x) for x in reviews['deceptive']]
train_indices, test_indices = [x for x in splitter.split(reviews['review'], labels)][cross_num]

training_X = np.array([reviews['review'][x] for x in train_indices])
training_y = np.array([labels[x] for x in train_indices])
test_X = np.array([reviews['review'][x] for x in test_indices])
test_y = np.array([labels[x] for x in test_indices])

# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = ["0", "1"]

In [ ]:
#Colummns are x (text), and y (labels)
train = pd.DataFrame(zip(training_X, training_y), columns=["x", "y"])
test = pd.DataFrame(zip(test_X, test_y), columns=["x", "y"])
train.y.value_counts(), test.y.value_counts()

(1    6509
 0    6365
 Name: y, dtype: int64, 1    1628
 0    1591
 Name: y, dtype: int64)

In [ ]:
test_review= train['x'][1]
test_label = train['y'][1]
print((type(test_review), type(test_label)))

(<class 'numpy.str_'>, <class 'numpy.str_'>)


In [ ]:
train["prefix"] = "Genuine Review?"
train= train.rename(
  columns={"x": "input_text", "y": "target_text"})

In [ ]:
# Model arguemnts
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128, #words in model vocabullary
    "train_batch_size": 16,
    "num_train_epochs": 1,
    "num_beams": None,
    "do_sample": True,
    "max_length": 1,
    "top_k": 1, #since this is binary, choose the top, https://jalammar.github.io/illustrated-gpt2/
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": True,
    "evaluate_during_training_verbose": True,
    "num_return_sequences": 1 #number of samples to generate
}
model = T5Model("t5","t5-small", args=model_args, use_cuda = False)

In [ ]:
import tensorflow as tf
# Calculate evaluation metrics.
def metric_fn(label_ids, predicted_labels):
    accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
    f1_score = tf.contrib.metrics.f1_score(
        label_ids,
        predicted_labels)
    auc = tf.metrics.auc(
        label_ids,
        predicted_labels)
    recall = tf.metrics.recall(
        label_ids,
        predicted_labels)
    precision = tf.metrics.precision(
        label_ids,
        predicted_labels)
    true_pos = tf.metrics.true_positives(
        label_ids,
        predicted_labels)
    true_neg = tf.metrics.true_negatives(
        label_ids,
        predicted_labels)
    false_pos = tf.metrics.false_positives(
        label_ids,
        predicted_labels)
    false_neg = tf.metrics.false_negatives(
        label_ids,
        predicted_labels)
    return {
        "eval_accuracy": accuracy,
        "f1_score": f1_score,
        "auc": auc,
        "precision": precision,
        "recall": recall,
        "true_positives": true_pos,
        "true_negatives": true_neg,
        "false_positives": false_pos,
        "false_negatives": false_neg}

In [ ]:
# Train the model
model.train_model(train, output_dir = "../bestmodel", eval_data=test, metrics=metric_fn)

In [ ]:
# Save the best model